# HW 1

This homework will expand on what we've learned so far about geopandas to allow us to work with real data. We'll analyze how conservation agriculture, a collection of sustainable agriculture practices that emphasizes minimal soil disturbance and constant soil cover, spread throughout Mexico thanks to a national extension effort. This homework will use techniques we learned in Thursday classes like making points, making geodataframes, performing spatial joins, and plotting. We'll also cover new topics like how to read in excel files, CSVs, and shapefiles, how to subset a dataframe, and checking for duplicated entries in a data set.

The first step is to download the data we'll use [here](https://data.cimmyt.org/dataset.xhtml?persistentId=hdl:11529/10548986). We want `1.-Farmer_Plot_Logbook_2012-2022_03.xlsx`. You'll need to fill out a little form agreeing to the terms of use before the download starts. Once you've downloaded it, I suggest moving it to this directory (`.../spatial_data_analysis/hw/`), though it's not strictly necessary. 

If you open this file in excel, google sheets, or similar software, you'll see that it's structured pretty similarly to the pandas dataframes we worked with in class: column names near the top with rows filled with data. To actually read it in as a pandas DataFrame, we'll use `pandas.read_excel`. This function does exactly what it says: it reads excel files into pandas DataFrames. [The documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html) shows a lot of options, but the relevant one for us is `header`, which lets us set the row that contains the column names. If you open the file, you'll see that the column names are not in the first row, which is pandas's default guess (`header=0`). Try reading in the file with the correct column names. Note that the path to the file will depend on where you saved it and whether you moved it to this directory (or another place).

In [ ]:
import pandas as pd

df = pd.read_excel(
    "", # path to the downloaded file on your computer
    header = # The row to choose for column names (0-indexed)
)

df

With that looking good, let's look at the columns to find the spatial information. There are too many columns to see them all in the preview output above, so let's use the `columns` attribute to look for coordinates.

In [ ]:
 # access the columns attribute

It looks like there are two columns representing latitude and longitude. So we have (most of) the information to put these in space! We don't exactly know the CRS (there are many that measure coordinates in degrees of latitude and longitude), but WGS84 (EPSG:4326) is a good guess to start with because it's the most common. Let's make a column that will hold the point. 

We don't want to do this one-by-one for tens of thousands of rows, so we'll use the `apply` method, which let's us apply a function to each row of the DataFrame. The other thing we'll make use of is anonymous functions. One way to think of what `apply` does is it applies a function that we give it to a dictionary where the keys are the column names and the values are the values for a given row, and it does that for every row and sticks the results together. We need to make a function that will take that dictionary and make a point using the relevant columns. Since this is a one-off case, we don't need to define a permanent function for this, we'll use an anonymous function (also called a lambda function). I'll walk you through how this works.

In [ ]:
import shapely.geometry as shg

# We're making a new column called 'point'
df['point'] = df.apply( # We're applying a function to each row of the DataFrame df
    lambda x: # lambda let's us define an anonymous function, one that will only be used here.
    shg.Point(x[''],x['']), # we're making a point. Here, the xs are referring to the dictionary for a given row
    # so for the first argument, we want the key to be the column that contains the east-west component of the coordinate,
    # and for the second argument we want the key to be the column that contains the north-south component
    axis=1 # Somewhat confusingly, we add axis=1 to apply to say "do this for each row"
)
df

Now let's see if we did this correctly! Let's make df into a GeoDataFrame and plot it. Like I said, we're just going to guess that it's in WGS84 (it is).

In [ ]:
import geopandas as gpd

gdf = gpd.GeoDataFrame(, geometry='', crs='') # Fill in the missing arguments
gdf.plot()

If you've done everything correctly you should see something that vaguely looks like Mexico! We can check this more explicitly by plotting it along with Mexico's boundary. Let's get that [directly from INEGI](https://www.inegi.org.mx/app/biblioteca/ficha.html?upc=702825217341), Mexico's national statistical agency. Once you download it, unzip it. The shapefile we want is in `/conjunto_de_datos/areas_geoestadisticas_municipales.shp`. Again, I recommend moving it to this directory, but it isn't necessary. Note that you need to move all of the files associated with the shapefile, not just the `.shp` one (i.e. the files with the same name but different extension).

Similar to how pandas can directly read in excel files (and CSVs), geopandas can read in shapefiles with the `geopandas.read_file` function. In this case, we just give it the file and it reads off the metadata things such as the column that contains the geometry information and the CRS. ([Documentation](https://geopandas.org/en/stable/docs/reference/api/geopandas.read_file.html))

In [ ]:
mex = gpd.read_file("") # path to your shapefile, particularly the file ending in .shp
mex.plot()

If you look at the axes of the plot, you can tell this isn't measured in degrees of latitude and longitude. Check the CRS.

In [ ]:
mex.crs # access the CRS attribute

Indeed, not measured in degrees and definitely not in WGS84. Let's put it into WGS84 so we can compare with our other data.

In [ ]:
mex = mex.to_crs("EPSG:4326") # transform the CRS to WGS84
mex.plot()

Now that we've got both our datasets in WGS84, let's plot them together. For this, we'll need to use `matplotlib.pyplot` to make sure we plot both of them at once.

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,1, figsize=(10,20)) # this says make a single plot (1 row of plots, and 1 column) that is 10x20 in size

mex.boundary.plot(ax = ax) #plotting the boundary attribute avoids filling in all of Mexico's interior, so we can see a bit better.
# The ax = ax part says to plot it in our single figure

gdf. # Try filling in this part

Again, if all has gone well we should see a bunch of overlapping dots inside of Mexico.

Now let's work up to the final goal of this homework: The total area of unique plots in each municipality that adopted conservation agriculture as part of this extension effort. There are a few steps here. First, not all plots adopt conservation agriculture, so we need to find the ones that do. Fortunately, there's a column that keeps track of this.

In [ ]:
gdf[''] #check the column that tracks conservation agriculture adoption

Here, 1 means adopted, 0 means did not adopt, and NaN is no data. Let's get only the rows that adopted. 

Pandas and geopandas do this similarly: we'll use `.loc`, which is kind of like a method (honestly, I actually don't know what to call it). The way it works is we use `dataframe.loc[condition]`, where `condition` is like a list with an element for each row that is either true or false, to select the rows where `condition==true`. A really common case, and the one we'll use here, is if we want to check if a column is equal to something: `dataframe.loc[dataframe['column name']==value]`. Let's try it here.

In [ ]:
ca_gdf = gdf.loc[gdf['']==] # Fill in with the relevant column name and value
ca_gdf

This should have a lot fewer rows than `gdf`. Now we want unique plots. For this, we'll choose a subset of columns and use the `drop_duplicates` method. The way we select a subset of columns (in both pandas and geopandas) is `dataframe[['column1','column2',...]]`, it's kind of like how we access a particular column, `dataframe['column']`, but now with a list of columns. The columns we need are:

- Something that tracks unique plots
- The area (there are two, use the "survey" one)
- The spatial information (the geometry, not the coordinates)

Try selecting those 3 columns and dropping duplicates.

In [ ]:
ca_gdf_dd = ca_gdf[['PLOT ID','TOTAL PLOT AREA (HA) DATA OBTAINED BY SURVEY WITH THE FARMER','point']].drop_duplicates()
ca_gdf_dd

(It turns out that this actually doesn't completely uniquely identify plots becase they are sometimes recorded as having different area or coordinates but you get the idea)

Now that we've got that, let's do a spatial join! In this case, we're going to want to keep the Mexican municipality geometries, and we want to keep all of them. This should be pretty similar to the spatial join we did in class.

In [ ]:
mun_ca = gpd.sjoin(
    , # Left geodataframe
    , # right geodataframe
    how = '' # Kind of join
)

mun_ca

And the last step before we plot: we need to sum up the area by municipality. We'll use the  `groupby` method, and then merge back on the spatial information. I'll walk you through this, but first we need to replace those `NaN`s with zeros using the `fillna` method because `NaN`s can cause use to drop rows when we aggregate.

In [ ]:
mun_ca[''] = mun_ca[''].fillna(0) # put the name of the area column in both
mun_ca

In [ ]:
'''
The list inside groupby should have the names of two columns that uniquely identify each municipality
The brackets after groupby are saying "do something to this column", so we want the area colum
the sum method is summing the column we selected within each group
the reset index method is because groupby sets the index to the groups. Try getting rid of it to see what I mean.
'''
mun_ca_tot = mun_ca.groupby([])[].sum().reset_index()
mun_ca_tot

Now we merge back on the spatial data. We'll do this with `pandas.merge`. Since we made sure we weren't dropping any municipalities we can do this as an inner join, which is the default. We use `on` to choose the keys for the join. One other thing about this is that pandas "forgets" that one of these is a geopandas GeoDataFrame during the join, so we have to remind it.

In [ ]:
mun_ca_tot = pd.merge(mun_ca_tot, mex,# Merge back on the spatial information
                     on=[]) # Using those two columns that uniquely identify  
mun_ca_tot = gpd.GeoDataFrame(mun_ca_tot,geometry='',crs="") # fill this in again
mun_ca_tot.plot(column='',legend=True, figsize=(10,20)) # The area column

And there we have it!